# Determining the crop coefficient function with Python

*M. Vremec, October 2022, University of Graz*

Data source: ZAMG - https://data.hub.zamg.ac.at

What is done:

- load the station data from ZAMG
- estimate potential evapotranspiration
- determine the crop coefficient function based on equation 65 in Allen et al. 1998
- plot and store result

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyet
pyet.show_versions()

## 1. Loading daily data from ZAMG (Messstationen Tagesdaten)

station: Graz Universität 16412

Selected variables:
- globalstrahlung (global radiation), J/cm2 needs to be in MJ/m3d, ZAMG abbreviation - strahl
- arithmetische windgeschwindigkeit (wind speed), m/s, ZAMG abbreviation - vv
- relative feuchte (relative humidity), %, ZAMG abbreviation - rel
- lufttemparatur (air temperature) in 2 m, C, ZAMG abbreviation - t
- lufttemperatur (air temperature) max in 2 m, C, ZAMG abbreviation - tmax
- lufttemperatur (air temperature) min in 2 m, C, ZAMG abbreviation - tmin
- latitute and elevation of a station

In [ ]:
#read data
data_16412 = pd.read_csv('data/example_1/klima_daily.csv', index_col=1, parse_dates=True)
data_16412

## 2. Calculate PET for Graz Universität - 16412

In [ ]:
# Convert Glabalstrahlung J/cm2 to MJ/m2 by dividing to 100

meteo = pd.DataFrame({"time":data_16412.index, "tmean":data_16412.t, "tmax":data_16412.tmax, "tmin":data_16412.tmin, "rh":data_16412.rel, 
                      "wind":data_16412.vv, "rs":data_16412.strahl/100})
time, tmean, tmax, tmin, rh, wind, rs = [meteo[col] for col in meteo.columns]

lat = 47.077778*np.pi/180  # Latitude of the meteorological station, converting from degrees to radians
elevation = 367  # meters above sea-level

# Estimate potential ET with Penman-Monteith FAO-56
pet_pm = pyet.pm_fao56(tmean, wind, rs=rs, elevation=elevation, 
                      lat=lat, tmax=tmax, tmin=tmin, rh=rh)

## 3. Determine the crop coefficient function

Based on: https://www.fao.org/3/x0490e/x0490e0b.htm
figure 34.


![Figure 34](https://www.fao.org/3/x0490e/x0490e6k.gif)

In [ ]:
Kcini = 0.3 
Kcmid = 1.1
Kcend = 0.65

crop_ini = pd.Timestamp("2020-04-01")
crop_dev = pd.Timestamp("2020-05-01")
mid_season = pd.Timestamp("2020-06-01")
late_s_start = pd.Timestamp("2020-07-01")
late_s_end = pd.Timestamp("2020-08-01")

In [ ]:
kc = pd.Series(index=[crop_ini, crop_dev, mid_season, late_s_start, late_s_end],
                 data=[Kcini, Kcini, Kcmid, Kcmid, Kcend])
kc = kc.resample("d").mean().interpolate()
kc.plot()

In [ ]:
petc = pet_pm.loc[crop_dev:late_s_end] * kc

## 4. Plot results

In [ ]:
pet_pm.loc[crop_dev:late_s_end].plot(label="Potential evapotranspiration")
petc.loc[crop_dev:late_s_end].plot(label="Potential crop evapotranspiration")